In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [2]:
txtFile=sc.textFile('../data/covtype.csv')
#Convert it into RDD of lists 
rdd=(txtFile.map(lambda x:x.split())
    .map(lambda x: x[0].strip("'").split(","))
    .map(lambda x:[float(v) for v in x])
    .map(lambda x: (x[-1]-1,x[0:-1])))

In [3]:
import itertools
soil_list =[]
for k in range(40):
    string = 'Soil_Type_' + str(k+1)
    soil_list.append(string)
WA_list =[]
for k in range(4):
    string = 'WA_' + str(k+1)
    WA_list.append(string)
names = [['Elevation'], ['Aspect'], ['Slope'], ['HDHyrdo'], ['VDHydro'], ['HDRoadways'], \
         ['9amHills'],['NoonHills'], ['3pmHills'], ['HDFirePoints'], WA_list,\
         soil_list, ['Cover_Type']]
columns = list(itertools.chain(*names))

In [4]:
import random

In [5]:
#selecting random features
m = 5 #No of features
indices = range(len(columns)-1)
random.shuffle(indices)
sampled_c = [columns[j] for j in indices[:m]]

In [6]:
#Code for implementing in rdd
%time sampled_rdd = rdd.map(lambda x: (x[0],[x[1][j] for j in indices[:m]]))

CPU times: user 109 µs, sys: 17 µs, total: 126 µs
Wall time: 326 µs


In [7]:
#Imports - move later to top
import numpy as np

In [8]:
#total counts of the data
t_count = rdd.count()

In [87]:
##Implementing the ID3 and testing the Entropy
test_index = 0
%time entrop_score = entropy_score(sampled_rdd,test_index)
%time entrop_score = entrop_score.take(1)[0][1]

CPU times: user 19.9 ms, sys: 5.2 ms, total: 25.1 ms
Wall time: 40 ms
CPU times: user 8.84 ms, sys: 2.15 ms, total: 11 ms
Wall time: 8 s


In [9]:
# Defined entropy as 1- sum(percentages squared)
def entropy_score(sampled_rdd,index):
    c=sampled_rdd.count()
    ent = sampled_rdd.map(lambda x:x[1][index]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[1]/c)*(x[1]/c)).reduce(lambda a,b:a+b)
    cats=sampled_rdd.map(lambda x:x[1][index]).distinct().collect()
    return 1-ent,cats

In [62]:
def best_attr(sampled_rdd,m):
    best_a=0
    best_s=100
    categories=[]
    for i in m:
        s,c=entropy_score(sampled_rdd,i)
        print i,s,c
        if s<=best_s:
            best_a=i
            best_s=s
            categories=c
    return best_a, categories

In [71]:
%time entropy_score(sampled_rdd,1)

CPU times: user 32.7 ms, sys: 10 ms, total: 42.7 ms
Wall time: 35.1 s


0.000361372999140519

In [29]:
%time new_rdd = sampled_rdd.map(lambda x:x[1][test_index]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[0],(-x[1]/t_count)*(np.log2(x[1]/t_count))))\
            .map(lambda x:(1,x[1])).reduceByKey(lambda a,b:a+b)

CPU times: user 14.6 ms, sys: 3.09 ms, total: 17.7 ms
Wall time: 31.9 ms


In [30]:
%time print get_all_scores(sampled_rdd,m).collect()

[(0, 0.16207143284308864), (3, 0.027547699424360673), (6, 0.0025077543955656055), (9, 0.0095463517593474476), (1, 0.14900598458760123), (4, 6.5981678852644112), (7, 11.375626917177499), (2, 0.71862713465838812), (5, 0.031695711359855931), (8, 7.2756316788852136)]
CPU times: user 159 ms, sys: 36 ms, total: 195 ms
Wall time: 1min 31s


In [11]:
def infoGain(sampled_rdd, attr=0, target_attr=0):
    subset_entropy = 0
    val_freq = sampled_rdd.map(lambda x:x[1][attr]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[0],x[1]/t_count))
    val_freq = val_freq.collectAsMap()
    classes = sampled_rdd.map(lambda x: x[1][attr]).distinct().collect()
    print 'numClasses:',len(classes)
    for cls in classes:
        sub_rdd=sampled_rdd.filter(lambda x: x[1][attr]==cls)
        try:
            if sub_rdd.isEmpty() == True:
                continue
        except:
            print "Error in calculation. Moving on"
            continue
        subs_entropy = entropy_score(sub_rdd,target_attr)
        entropy = subs_entropy.collect()
        entropy = entropy[0][1]
        subset_entropy += entropy*val_freq[cls]
    return entropy_score(sampled_rdd,target_attr).collect()[0][1] - subset_entropy

In [12]:
from collections import Counter
import numpy as np
def IG(l):
    length=len(l)
    c=Counter()
    for v in l:
        c[v] += 1.0/length
    return 1-sum(np.multiply(c.values(),c.values()))

In [25]:
def infoGain2(sampled_rdd,attr=0):
    c=sampled_rdd.count()
    pair_rdd=sampled_rdd.map(lambda x: (x[1][attr],x[0]))
    list_rdd=pair_rdd.groupByKey().mapValues(lambda x: tuple(x)).map(lambda x: (x[0],x[1], len(x[1])/float(c)))
    gains_rdd=list_rdd.map(lambda x: (x[0],IG(x[1]),x[2]))
    return gains_rdd.map(lambda x: x[1]*x[2]).reduce(lambda a,b:a+b)

In [26]:
%time infoGain2(sampled_rdd, attr=0)

CPU times: user 18.9 ms, sys: 4.26 ms, total: 23.2 ms
Wall time: 16.2 s


0.62237568821799927

In [31]:
def choose_attribute(sampled_rdd, attributes):
    best_gain = 100.0
    best_attr = None
    for att in attributes:
        gain = infoGain2(sampled_rdd,att)
        print att,gain
        if gain <= best_gain:
            best_gain = gain
            best_attr = att
    return best_gain,best_attr

In [38]:
%time best_attr(sample,4)

0 0.53125 [0, 1, 2]
1 0.59375 [0, 1, 2]
2 0.59375 [0, 1, 2]
3 0.40625 [0, 1, 2]
CPU times: user 134 ms, sys: 32.7 ms, total: 166 ms
Wall time: 1.22 s


(3, [0, 1, 2])

In [32]:
sample = sc.parallelize([(1,[1,1,0,1]),
         (0,[0,0,1,1]),
         (1,[1,1,0,2]),
         (0,[1,1,0,1]),
         (0,[0,0,1,0]),
         (1,[1,0,2,1]),
         (1,[2,0,1,1]),
         (1,[1,2,1,1])])

In [33]:
choose_attribute(sample,range(4))

0 0.2
1 0.416666666667
2 0.416666666667
3 0.333333333333


(0.1999999999999999, 0)

In [21]:
# get best starting attribute
start_split,cats=best_attr(sample,4)

0 0.53125 [0, 1, 2]
1 0.59375 [0, 1, 2]
2 0.59375 [0, 1, 2]
3 0.40625 [0, 1, 2]


In [64]:
def createDecisionTree(sub_rdd,attributes):
    if len(attributes)-1 <= 0:
        return 0
    else:
        bestAttr,vals = best_attr(sub_rdd,attributes)
        print bestAttr
        tree = {bestAttr:{}}
        for val in vals:
            new_rdd = sub_rdd.filter(lambda x:x[1][bestAttr] == val)\
            .map(lambda x:(x[0],x[1]))
            print attributes
            attributes.remove(bestAttr)
            subtree = createDecisionTree(new_rdd,attributes)
            
            tree[bestAttr][val] = subtree
    return tree

In [61]:
createDecisionTree(sample,range(4))

0 0.53125 [0, 1, 2]
1 0.59375 [0, 1, 2]
2 0.59375 [0, 1, 2]
3 0.40625 [0, 1, 2]
0 []


{0: {}}

In [ ]:
##
#separate branches

# send subtrees to get attributes, recursively
# stop tree once you reach a point of 0 info gain
def recursive(sub_rdd,attr,target_attrs):
    best_attr, cats=choose_attribute(sub_rdd,attr,target_attrs)
    for cat in cats:
        target_attrs=#create a list of attributes you haven't checked yet
        b,c=recursive(sub_rdd.filter(lambda x:x[1][best_attr]==cat),best_attr,target_attrs)
        if b==0: # no new information
            return c



The first cut is calculated with best_attr. Subsequent cuts can be calucluated with choose_attribute.

ParallelCollectionRDD[221] at parallelize at PythonRDD.scala:391